## 2018 Choropleth of Segregation in Chicago Public Schools

Scroll all the way down to see interactive choropleth 

In [69]:
import pandas as pd
import numpy as np
import folium

In [89]:
profile_2018 = pd.read_csv('Chicago_Public_Schools_-_School_Profile_Information_SY1718.csv')
header = ['School_ID', 'School_Name', 'Network', 'Total', 'White_Student_Count', 
          'White_Student_Pct', 'African_American_Count', 'African_America_Pct', 
         'AAPI_Count', 'AAPI_Pct', 'Native_American_Count', 'Native_American_Pct',
          'Hispanic_Student_Count', 'Hispanic_Student_Pct', 
          'Multi-Racial_Student_Count', 'Multi-Racial_Student_Pct', 
          'Asian_Student_Count', 'Asian_Student_Pct', 'Hawaiian_PI_Count', 
          'Hawaiian_PI_Pct', 'Not_Available_Count', 'Not_Available_Pct']
demo_data_2018 = pd.read_excel('Demographics_RacialEthnic_2018.xls', 'Schools', skiprows=2, names = header)
demo_data_2018['School_ID'].astype(int, inplace=True);
profile_merge = pd.merge(demo_data_2018[['Network', 'School_ID']], profile_2018, how='inner', on='School_ID')

grouped_1 = profile_merge.groupby('Network_x')
network_totals = grouped_1['Student_Count_Total'].sum()
black_student_totals = grouped_1['Student_Count_Black'].sum()
hispanic_student_totals = grouped_1['Student_Count_Hispanic'].sum()
white_student_totals = grouped_1['Student_Count_White'].sum()
asian_student_totals = grouped_1['Student_Count_Asian'].sum()
native_american_totals = grouped_1['Student_Count_Native_American'].sum()
hawaiian_pi_totals = grouped_1['Student_Count_Hawaiian_Pacific_Islander'].sum()
asian_pi_totals = grouped_1['Student_Count_Asian_Pacific_Islander'].sum()
other_ethnicity_totals = grouped_1['Student_Count_Other_Ethnicity'].sum()
multi_totals = grouped_1['Student_Count_Multi'].sum()
na_totals = grouped_1['Student_Count_Ethnicity_Not_Available'].sum()

df = {'Network_Totals': network_totals, 'Black_Student_Totals': black_student_totals,
     'Hispanic_Student_Totals': hispanic_student_totals, 'White_Student_Totals': 
      white_student_totals, 'Asian_Student_Totals': asian_student_totals, 
     'Native_American_Student_Totals': native_american_totals, 'Hawaiian_Pacific_Islander_Totals':
     hawaiian_pi_totals, 'Asian_Pacific_Islander_Totals': asian_pi_totals, 'Other_Ethnicity_Totals':
     other_ethnicity_totals, 'Multi_Ethnicity_Totals': multi_totals, 
      'Ethnicity_NA_Totals': na_totals}
df = pd.DataFrame(df)

df.drop('Other_Ethnicity_Totals', axis=1, inplace=True)
df['Totals-Ethnicity_NA'] = df['Network_Totals']-df['Ethnicity_NA_Totals']
df.drop('Ethnicity_NA_Totals', axis=1, inplace=True)

row_total = []
test = df['Totals-Ethnicity_NA']
for n in range(len(df.index)):
    network_total = df.iloc[n, 9]
    x = df.iloc[n, 1:9]
    row_total.append(sum([(j/network_total)*np.log(1/(j/network_total)) for j in x if j!=0]))
df['Network_Entropy'] = np.array(row_total)

profile_merge['Network_Entropy'] = profile_merge['Network_x']
network_1 = df.loc['Network 1', 'Network_Entropy']
network_2 = df.loc['Network 2', 'Network_Entropy']
network_3 = df.loc['Network 3', 'Network_Entropy']
network_4 = df.loc['Network 4', 'Network_Entropy']
network_5 = df.loc['Network 5', 'Network_Entropy']
network_6 = df.loc['Network 6', 'Network_Entropy']
network_7 = df.loc['Network 7', 'Network_Entropy']
network_8 = df.loc['Network 8', 'Network_Entropy']
network_9 = df.loc['Network 9', 'Network_Entropy']
network_10 = df.loc['Network 10', 'Network_Entropy']
network_11 = df.loc['Network 11', 'Network_Entropy']
network_12 = df.loc['Network 12', 'Network_Entropy']
network_13 = df.loc['Network 13', 'Network_Entropy']
ausl = df.loc['AUSL', 'Network_Entropy']
charter = df.loc['Charter', 'Network_Entropy']
contract = df.loc['Contract', 'Network_Entropy']
isp = df.loc['ISP', 'Network_Entropy']
options = df.loc['Options', 'Network_Entropy']

mapdict = {'Network 1': network_1, 'Network 2': network_2, 'Network 3': network_3,
          'Network 4': network_4, 'Network 5': network_5, 'Network 6': network_6,
          'Network 7': network_7, 'Network 8': network_8, 'Network 9': network_9, 
          'Network 10': network_10, 'Network 11': network_11, 'Network 12': network_12,
          'Network 13': network_13, 'AUSL': ausl, 'Charter': charter, 'Contract': contract, 
          'ISP': isp, 'Options':options}
profile_merge['Network_Entropy'] = profile_merge['Network_Entropy'].map(mapdict)

geo_markers = profile_merge[profile_merge['Network_x'] == 'Charter'][['Short_Name', 'School_Latitude', 'School_Longitude']]

profile_merge.drop('Student_Count_Other_Ethnicity', axis=1, inplace=True)
profile_merge['Adjusted_Counts'] = profile_merge['Student_Count_Total'] - profile_merge['Student_Count_Ethnicity_Not_Available']
profile_merge.drop('Student_Count_Ethnicity_Not_Available', axis=1, inplace=True)

row_total = []
for i in range(len(profile_merge.index)):
    school_total = profile_merge.iloc[i, 92]
    x = profile_merge.iloc[i, 31:39]
    row_total.append(sum([(i/school_total)*np.log(1/(i/school_total)) for i in x if i!=0]))
    
profile_merge['School_Entropy'] = np.array(row_total)
profile_merge['District_Student_Count'] = profile_merge['Network_x']

count_1 = df.loc['Network 1', 'Totals-Ethnicity_NA']
count_2 = df.loc['Network 2', 'Totals-Ethnicity_NA']
count_3 = df.loc['Network 3', 'Totals-Ethnicity_NA']
count_4 = df.loc['Network 4', 'Totals-Ethnicity_NA']
count_5 = df.loc['Network 5', 'Totals-Ethnicity_NA']
count_6 = df.loc['Network 6', 'Totals-Ethnicity_NA']
count_7 = df.loc['Network 7', 'Totals-Ethnicity_NA']
count_8 = df.loc['Network 8', 'Totals-Ethnicity_NA']
count_9 = df.loc['Network 9', 'Totals-Ethnicity_NA']
count_10 = df.loc['Network 10', 'Totals-Ethnicity_NA']
count_11 = df.loc['Network 11', 'Totals-Ethnicity_NA']
count_12 = df.loc['Network 12', 'Totals-Ethnicity_NA']
count_13 = df.loc['Network 13', 'Totals-Ethnicity_NA']
count_ausl = df.loc['AUSL', 'Totals-Ethnicity_NA']
count_charter = df.loc['Charter', 'Totals-Ethnicity_NA']
count_contract = df.loc['Contract', 'Totals-Ethnicity_NA']
count_isp = df.loc['ISP', 'Totals-Ethnicity_NA']
count_options = df.loc['Options', 'Totals-Ethnicity_NA']

mapdict_count = {'Network 1': count_1, 'Network 2': count_2, 'Network 3': count_3,
          'Network 4': count_4, 'Network 5': count_5, 'Network 6': count_6,
          'Network 7': count_7, 'Network 8': count_8, 'Network 9': count_9, 
          'Network 10': count_10, 'Network 11': count_11, 'Network 12': count_12,
          'Network 13': count_13, 'AUSL': count_ausl, 'Charter': count_charter, 'Contract': count_contract, 
          'ISP': count_isp, 'Options': count_options}
profile_merge['District_Student_Count'] = profile_merge['District_Student_Count'].map(mapdict_count)

profile_merge['School_Theil'] = (profile_merge['Adjusted_Counts']*(profile_merge['Network_Entropy']-profile_merge['School_Entropy'])
                    /(profile_merge['Network_Entropy']*profile_merge['District_Student_Count']))

grouped_net = profile_merge.groupby('Network_x')
network_theil = pd.DataFrame(grouped_net['School_Theil'].sum())
theil_13 = network_theil.reset_index()
theil_13 = theil_13.drop([0, 1, 2, 3, 17])
theil_13['Network_x'] = theil_13['Network_x'].str.replace('Network ', '')
network_theil

,School_Theil
Network_x,
AUSL,0.487139
Charter,0.562936
Contract,0.218191
ISP,0.408899
Network 1,0.151295
Network 10,0.434947
Network 11,0.228018
Network 12,0.254647
Network 13,0.620280


In [90]:
schools_map = folium.Map(location=[41.8, -87.54],
                   tiles='Mapbox Bright', zoom_start=10)
schools_map.choropleth(geo_data='Chicago Public Schools - Geographic Networks (1).geojson',
                      name='choropleth',
 data= theil_13,
 columns=['Network_x', 'School_Theil'],
 key_on='feature.properties.network_num',
 fill_color='PuRd',
 fill_opacity=0.4,
 line_opacity=1,
 legend_name='Theil Index'
)

for i in range(0,len(geo_markers)):
    folium.CircleMarker([geo_markers.iloc[i]['School_Latitude'], geo_markers.iloc[i]['School_Longitude']], popup=geo_markers.iloc[i]['Short_Name'], radius=4, color='black', fill_color='darkred', prefix='fa').add_to(schools_map)

legend_html = '''
    
     <div style="position: fixed; 
     bottom: 20px; left: 20px; width: 250px; height: 70px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     "">&nbsp; <b> Marker Legend </b> <br>
     &nbsp; Charter School Location (2018) &nbsp; 
    <i class="fa fa-circle-o"
             style=”color:black”></i><br>
      </div>
     '''

schools_map.get_root().html.add_child(folium.Element(legend_html))
schools_map

The above map shows the Theil index of each public network boundary. The markers show the Theil index and locations of charter schools. 